In [48]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import numpy as np
from sdf import *
from utils.external_import import igl
import sdf

@sdf3
def gyroid(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return (np.cos(x)*np.sin(y) + np.cos(y)*np.sin(z) + np.cos(z)*np.sin(x) - t)
    return f

box(1.0) & gyroid(w=math.pi*4, t=0)

class ImplicitDataset():
    def __init__(
        self, f, N=1000, scale_offset=0.1,
        output_stl='tmp.stl', device=None, from_file:str=None, from_dataset:dict=None,
        *args, **vargs
    ):
        _ndim = round(N**(1/3))
        if _ndim <= 0:
            raise ValueError('N must be greater than or equal to 1')
        # Generate grid (x,y,z) from cartesian product
        (x0, y0, z0), (x1, y1, z1) = sdf.mesh._estimate_bounds(f)
        offset_x = abs(scale_offset*(x0-x1))
        offset_y = abs(scale_offset*(y0-y1))
        offset_z = abs(scale_offset*(z0-z1))
        assert(x0 <= x1 and y0 <= y1 and z0 <= z1 and offset_x >= 0 and offset_y >= 0 and offset_z >= 0)

        X = np.linspace(x0-offset_x, x1+offset_x, _ndim, dtype=np.float32)
        Y = np.linspace(y0-offset_y, y1+offset_y, _ndim, dtype=np.float32)
        Z = np.linspace(y0-offset_z, y1+offset_z, _ndim, dtype=np.float32)
        self.points = sdf.mesh._cartesian_product(X, Y, Z)
        
        lvl_set = f(self.points).reshape((_ndim, _ndim, _ndim))
        dx = (X[1] - X[0], Y[1] - Y[0], Z[1] - Z[0])
        import skfmm
        sdfs = skfmm.distance(lvl_set, dx=dx, periodic=[True, True, True])
        grad = np.array(np.gradient(sdfs, *dx))
        self.sdfs = sdfs.reshape((_ndim**3,)).squeeze()
        self.grads = grad.reshape((3, _ndim**3)).T

        try:
            temp_filename = os.path.normpath(output_stl)
            sdf.write_binary_stl(temp_filename, f.generate(step=dx[0], verbose=False, method=1))
            # Load mesh
            v,f = igl.read_triangle_mesh(temp_filename)
            _, _, _, self.true_normals = igl.signed_distance(self.points, v, f, 0, return_normals=True)
            self.true_sdfs, _, _ = igl.signed_distance(self.points, v, f, 4)
            self.true_grads = np.array(np.gradient(self.true_sdfs.reshape((_ndim, _ndim, _ndim)), *dx))
            self.true_grads = self.true_grads.reshape((3, _ndim**3)).T

        except Exception as e:
            print("warning: ", e)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
N_train=40*40*40
N_test=1e6
name='box_1f0_gyroid_4pi'
save_dir='../datasets'
output_stl = os.path.join(save_dir, name + '.stl')

train_dataset = ImplicitDataset(
    f=box(1.0) & gyroid(w=math.pi*4, t=0), 
    N=int(N_train), scale_offset=0.1, output_stl=output_stl
)

In [51]:
norm_p = train_dataset.true_grads / np.linalg.norm(train_dataset.true_grads, axis=1).reshape(-1, 1)

In [52]:
describe(norm_p - train_dataset.true_normals)

DescribeResult(nobs=64000, minmax=(array([-1.39565557, -1.39185685, -1.52181009]), array([1.39685536, 1.54212218, 1.54354161])), mean=array([-0.01510923, -0.01396228, -0.0017386 ]), variance=array([0.06630257, 0.06471593, 0.06694999]), skewness=array([-0.27682884, -0.27383255, -0.13487733]), kurtosis=array([3.46693667, 3.51037346, 3.49661017]))

In [55]:
np.einsum('ij,ij->i',norm_p, train_dataset.true_normals)

array([0.93254797, 0.91516209, 0.94604783, ..., 0.88771478, 0.85966961,
       0.83834951])